In [ ]:
import numpy as np
import scipy
from collections import defaultdict

In [2]:
live_cells_time_series = np.loadtxt('simulation_data/interfering-blinker/live_cell_history.txt', dtype=np.int64)
print(live_cells_time_series)

[  6   7   8   9  13  16  13  19  16  16  19  22  21  27  30  43  25  20
  19  16  14  10  10  15  14  18  20  28  24  24  33  26  33  29  30  28
  33  31  42  31  35  34  41  36  32  26  35  29  29  30  24  28  25  39
  25  35  25  22  16  14  13  16  15  15  17  17  21  21  30  23  33  25
  22  22  24  28  26  36  30  46  38  60  38  41  44  57  38  42  27  23
  24  22  24  28  31  37  39  38  38  48  37  38  44  54  46  62  54  53
  64  69  74  82  81  86  83  83  79  95  92  92  99  97  97 121 103 122
 119 132 115 162 140 152 164 133 119 103 121  99 111 109  97 117 115 113
 113 103 101  99  95  97  97  89 109 101 137 105 123 143 163 171 165 183
 153 165 142 150 142 138 136 128 120 116 108  98  90 100  92 100  86  92
  92  80  90  88  88  86  84 106  94 126 114 132 138 164 142 170 164 166
 130 126 148 116 134 100 108 114 124 120 104 102 124  98 118 110 118 130
 110 132 114  98 100 104  88  82  90  82  92  70  68  58  78  66  72  62
  58  58  50  52  54  46  42  42  46  46  46  46  4

In [3]:
def basic_statistics(time_series):
    stats_dict = {
        'mean': np.mean(time_series),
        'variance': np.var(time_series),
        'std_dev': np.std(time_series)
    }
    return stats_dict

def create_transition_matrix(time_series):
    """
    Create first-order transition matrix from observed transitions
    
    Parameters:
    time_series: numpy array of the sum at each time step
    
    Returns:
    dict: Transition probabilities and unique states
    """
    # Count transitions
    transitions = defaultdict(lambda: defaultdict(int))
    unique_states = sorted(set(time_series))
    state_to_idx = {state: idx for idx, state in enumerate(unique_states)}
    
    # Count occurrences of each transition
    for t in range(len(time_series)-1):
        current = time_series[t]
        next_state = time_series[t+1]
        transitions[current][next_state] += 1
    
    # Create and fill transition matrix
    n_states = len(unique_states)
    trans_matrix = np.zeros((n_states, n_states))
    
    for i, state_i in enumerate(unique_states):
        total = sum(transitions[state_i].values())
        if total > 0:  # Avoid division by zero
            for j, state_j in enumerate(unique_states):
                trans_matrix[i,j] = transitions[state_i][state_j] / total
    
    return {
        'transition_matrix': trans_matrix,
        'states': unique_states,
        'state_to_idx': state_to_idx
    }

def test_markov_property(time_series):
    """
    Test for the Markov property by comparing conditional probabilities
    P(X_t+1 | X_t) vs P(X_t+1 | X_t, X_t-1)
    
    Parameters:
    time_series: numpy array of the sum at each time step
    
    Returns:
    dict: Test results and transition probabilities
    """
    # Count first-order transitions
    first_order = defaultdict(lambda: defaultdict(int))
    for t in range(len(time_series)-1):
        current = time_series[t]
        next_state = time_series[t+1]
        first_order[current][next_state] += 1
    
    # Count second-order transitions
    second_order = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    for t in range(len(time_series)-2):
        prev = time_series[t]
        current = time_series[t+1]
        next_state = time_series[t+2]
        second_order[prev][current][next_state] += 1
    
    # Calculate chi-square statistics comparing distributions
    chi2_stats = []
    p_values = []
    
    for prev in set(time_series[:-2]):
        for current in set(time_series[1:-1]):
            # Get first-order distribution
            total_first = sum(first_order[current].values())
            if total_first == 0:
                continue
                
            first_dist = np.array([first_order[current][next_state] / total_first 
                                 for next_state in set(time_series[2:])])
            
            # Get second-order distribution
            total_second = sum(second_order[prev][current].values())
            if total_second == 0:
                continue
                
            second_dist = np.array([second_order[prev][current][next_state] / total_second 
                                  for next_state in set(time_series[2:])])
            
            # Only compare if we have enough observations
            if total_first > 5 and total_second > 5:
                chi2, p = stats.chisquare(second_dist, first_dist)
                chi2_stats.append(chi2)
                p_values.append(p)
    
    return {
        'chi2_mean': np.mean(chi2_stats) if chi2_stats else None,
        'p_value_mean': np.mean(p_values) if p_values else None,
        'n_comparisons': len(chi2_stats),
        'first_order_transitions': dict(first_order),
        'transition_matrix': create_transition_matrix(time_series)
    }

In [4]:
tm = create_transition_matrix(live_cells_time_series)['transition_matrix']
# print(create_transition_matrix(live_cells_time_series)['transition_matrix'])
for row in tm:
    for transition_probability in row:
        if abs(transition_probability - 0) < 1e-6 or abs(transition_probability - 1) < 1e-6:
            print(f'{int(transition_probability):^6}', end='')  # If the probability is 0 or 1, print it with no decimals
        else:
            print(f'{transition_probability:^6.3f}', end='')  # any other probability gets 3 decimal places
    print() # print a new line at the end of a distributiuon


  0     1     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
  0     0     1     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0

In [5]:
stats = basic_statistics(live_cells_time_series)
print("Basic statistics:", stats)

markov_test = test_markov_property(live_cells_time_series)
print("Markov property test results:")
print(f"Average chi-square statistic: {markov_test['chi2_mean']}")
print(f"Average p-value: {markov_test['p_value_mean']}")
print(f"Number of state transitions compared: {markov_test['n_comparisons']}")

Basic statistics: {'mean': np.float64(67.4135593220339), 'variance': np.float64(1833.6594771617354), 'std_dev': np.float64(42.82125029890808)}


AttributeError: 'dict' object has no attribute 'chisquare'

In [ ]:
from collections import Counter

In [ ]:
def analyze_state_repetition(time_series):
    unique_states = set(time_series)
    state_counts = Counter(time_series)
    transitions = defaultdict(list)
    
    for t in range(len(time_series)-1):
        current = time_series[t]
        next_state = time_series[t+1]
        transitions[current].append(next_state)
    
    print(f"Total time steps: {len(time_series)}")
    print(f"Unique states observed: {len(unique_states)}")
    print(f"States that appear more than once: {sum(1 for count in state_counts.values() if count > 1)}")
    print(f"States with multiple different next states: {sum(1 for next_states in transitions.values() if len(set(next_states)) > 1)}")
    
    return state_counts, transitions

In [ ]:
state_counts, transitions = analyze_state_repetition(live_cells_time_series)

Total time steps: 295
Unique states observed: 117
States that appear more than once: 65
States with multiple different next states: 65


In [ ]:
def analyze_markov_property(transitions, state_counts):
    # For states with multiple occurrences, analyze their next-state distributions
    markov_analysis = {}
    
    for state in transitions:
        if state_counts[state] >= 4:  # Only analyze states with enough data
            next_states = transitions[state]
            next_state_dist = Counter(next_states)
            
            # Calculate entropy of next-state distribution
            total = len(next_states)
            entropy = -sum((count/total) * np.log2(count/total) 
                         for count in next_state_dist.values())
            
            markov_analysis[state] = {
                'entropy': entropy,
                'unique_next_states': len(next_state_dist),
                'transitions': dict(next_state_dist)
            }
    
    return markov_analysis

In [ ]:
def print_markov_analysis(markov_analysis):
    for state in sorted(markov_analysis.keys()):
        info = markov_analysis[state]
        print(f"\nState {int(state)}:")
        print(f"  Entropy: {info['entropy']:.3f}")
        print(f"  Number of unique next states: {info['unique_next_states']}")
        print("  Transitions:")
        for next_state, count in info['transitions'].items():
            print(f"    -> {int(next_state)}: {count} times")

In [ ]:
print_markov_analysis(analyze_markov_property(transitions,state_counts))


State 16:
  Entropy: 2.252
  Number of unique next states: 5
  Transitions:
    -> 13: 1 times
    -> 16: 1 times
    -> 19: 1 times
    -> 14: 2 times
    -> 15: 1 times

State 22:
  Entropy: 1.922
  Number of unique next states: 4
  Transitions:
    -> 21: 1 times
    -> 16: 1 times
    -> 22: 1 times
    -> 24: 2 times

State 24:
  Entropy: 1.792
  Number of unique next states: 4
  Transitions:
    -> 24: 1 times
    -> 33: 1 times
    -> 28: 3 times
    -> 22: 1 times

State 25:
  Entropy: 1.922
  Number of unique next states: 4
  Transitions:
    -> 20: 1 times
    -> 39: 1 times
    -> 35: 1 times
    -> 22: 2 times

State 28:
  Entropy: 2.322
  Number of unique next states: 5
  Transitions:
    -> 24: 1 times
    -> 33: 1 times
    -> 25: 1 times
    -> 26: 1 times
    -> 31: 1 times

State 30:
  Entropy: 2.322
  Number of unique next states: 5
  Transitions:
    -> 43: 1 times
    -> 28: 1 times
    -> 24: 1 times
    -> 23: 1 times
    -> 46: 1 times

State 33:
  Entropy: 2.0

In [ ]:
def analyze_markov_property(transitions, state_counts):
    """
    Analyze Markov property using entropy and transition patterns
    
    Parameters:
    transitions: dict of lists containing observed transitions from each state
    state_counts: Counter object with counts of each state
    
    Returns:
    dict: Analysis results including entropy and transition patterns
    """
    markov_analysis = {}
    
    # Maximum possible entropy given number of unique next states
    def max_entropy(n):
        return np.log2(n)  # Equal probability distribution maximizes entropy
    
    for state in transitions:
        if state_counts[state] >= 4:  # Only analyze states with enough data
            next_states = transitions[state]
            next_state_dist = Counter(next_states)
            
            # Calculate entropy of next-state distribution
            total = len(next_states)
            entropy = -sum((count/total) * np.log2(count/total) 
                         for count in next_state_dist.values())
            
            # Calculate maximum possible entropy for this number of next states
            max_possible_entropy = max_entropy(len(next_state_dist))
            
            # Calculate normalized entropy (0 to 1 scale)
            normalized_entropy = entropy / max_possible_entropy if max_possible_entropy > 0 else 0
            
            markov_analysis[state] = {
                'entropy': entropy,
                'normalized_entropy': normalized_entropy,
                'max_possible_entropy': max_possible_entropy,
                'unique_next_states': len(next_state_dist),
                'number_of_transitions': total,
                'transitions': dict(next_state_dist),
                'transition_probabilities': {
                    next_state: count/total 
                    for next_state, count in next_state_dist.items()
                }
            }
    
    # Sort states by normalized entropy for easier analysis
    sorted_states = sorted(
        markov_analysis.keys(),
        key=lambda x: markov_analysis[x]['normalized_entropy'],
        reverse=True
    )
    
    return markov_analysis, sorted_states

print("\nDetailed Markov Analysis:")
markov_analysis, sorted_states = analyze_markov_property(transitions, state_counts)

print("\nStates ordered by normalized entropy (most random to most deterministic):")
for state in sorted_states:
    analysis = markov_analysis[state]
    print(f"\nState {state}:")
    print(f"Normalized Entropy: {analysis['normalized_entropy']:.3f}")
    print(f"Raw Entropy: {analysis['entropy']:.3f} bits")
    print(f"Maximum Possible Entropy: {analysis['max_possible_entropy']:.3f} bits")
    print(f"Number of unique next states: {analysis['unique_next_states']}")
    print(f"Total transitions observed: {analysis['number_of_transitions']}")
    print("Transition probabilities:")
    for next_state, prob in analysis['transition_probabilities'].items():
        print(f"  -> {next_state}: {prob:.3f}")


Detailed Markov Analysis:

States ordered by normalized entropy (most random to most deterministic):

State 30:
Normalized Entropy: 1.000
Raw Entropy: 2.322 bits
Maximum Possible Entropy: 2.322 bits
Number of unique next states: 5
Total transitions observed: 5
Transition probabilities:
  -> 43: 0.200
  -> 28: 0.200
  -> 24: 0.200
  -> 23: 0.200
  -> 46: 0.200

State 28:
Normalized Entropy: 1.000
Raw Entropy: 2.322 bits
Maximum Possible Entropy: 2.322 bits
Number of unique next states: 5
Total transitions observed: 5
Transition probabilities:
  -> 24: 0.200
  -> 33: 0.200
  -> 25: 0.200
  -> 26: 0.200
  -> 31: 0.200

State 33:
Normalized Entropy: 1.000
Raw Entropy: 2.000 bits
Maximum Possible Entropy: 2.000 bits
Number of unique next states: 4
Total transitions observed: 4
Transition probabilities:
  -> 26: 0.250
  -> 29: 0.250
  -> 31: 0.250
  -> 25: 0.250

State 42:
Normalized Entropy: 1.000
Raw Entropy: 2.000 bits
Maximum Possible Entropy: 2.000 bits
Number of unique next states: 4
